# Installation

In [ ]:
# Required imports. Please execute this cell first.
%pip install -q "tensorflow-macos>=2.5; sys_platform == 'darwin' and platform_machine == 'arm64'"
%pip install -q "tensorflow>=2.5; sys_platform == 'darwin' and platform_machine != 'arm64'"
%pip install -q "tensorflow>=2.5; sys_platform != 'darwin'"
%pip install -q --no-deps "tensorflow-hub"
%pip install -q "openvino>=2024.4.0" "nncf>=2.9.0" "requests" "transformers>=4.31" "onnx!=1.16.2" "tf_keras"
%pip install -q "torch>=2.1" "torchvision>=0.16" "ultralytics==8.3.59" onnx tqdm opencv-python --extra-index-url https://download.pytorch.org/whl/cpu

# Model Export

In [ ]:
import requests
from pathlib import Path

if not Path("notebook_utils.py").exists():
    r = requests.get(
        url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/notebook_utils.py",
    )
    open("notebook_utils.py", "w").write(r.text)

from notebook_utils import download_file, VideoPlayer, device_widget, quantization_widget



In [ ]:
from ultralytics import YOLO

DET_MODEL_NAME = "yolov8n"

det_model = YOLO("yolov8n.pt")

# Export the model to onnx
det_model.export(format="onnx", opset=17, simplify=True, dynamic=False, imgsz=640)

In [ ]:
import openvino as ov

# ov.convert_model returns an openvino.runtime.Model object
ONNX_CV_MODEL_PATH = "/content/yolov8n.onnx"
ov_model = ov.convert_model(ONNX_CV_MODEL_PATH)

# then model can be serialized to *.xml & *.bin files
# By default, the model will be saved as fp16 type. But check if your CPU supports FP16 operations, if not it will add additional overhead.
ov.save_model(ov_model, "/content/models/yolov8n_openvino_model/yolov8n.xml",compress_to_fp16=False)

# Static Quantization using nncf

This code uses Ultralytics to define the calibration data loader for quantization. However, you can create a custom DataLoader using PyTorch if needed.Refer to the PyTorch documentation for details on how to create a DataLoader.

In [ ]:
from zipfile import ZipFile

from ultralytics.data.utils import DATASETS_DIR


DATA_URL = "http://images.cocodataset.org/zips/val2017.zip"
LABELS_URL = "https://github.com/ultralytics/yolov5/releases/download/v1.0/coco2017labels-segments.zip"
CFG_URL = "https://raw.githubusercontent.com/ultralytics/ultralytics/v8.1.0/ultralytics/cfg/datasets/coco.yaml"

OUT_DIR = DATASETS_DIR

DATA_PATH = OUT_DIR / "val2017.zip"
LABELS_PATH = OUT_DIR / "coco2017labels-segments.zip"
CFG_PATH = OUT_DIR / "coco.yaml"

if not (OUT_DIR / "coco/labels").exists():
    download_file(DATA_URL, DATA_PATH.name, DATA_PATH.parent)
    download_file(LABELS_URL, LABELS_PATH.name, LABELS_PATH.parent)
    download_file(CFG_URL, CFG_PATH.name, CFG_PATH.parent)
    with ZipFile(LABELS_PATH, "r") as zip_ref:
        zip_ref.extractall(OUT_DIR)
    with ZipFile(DATA_PATH, "r") as zip_ref:
        zip_ref.extractall(OUT_DIR / "coco/images")

In [ ]:
from ultralytics.utils import DEFAULT_CFG
from ultralytics.cfg import get_cfg
from ultralytics.data.converter import coco80_to_coco91_class
from ultralytics.data.utils import check_det_dataset

args = get_cfg(cfg=DEFAULT_CFG)
args.data = str(CFG_PATH)

In [ ]:
det_validator = det_model.task_map[det_model.task]["validator"](args=args)

In [ ]:
det_validator.data = check_det_dataset(args.data)
det_validator.stride = 32
det_data_loader = det_validator.get_dataloader(OUT_DIR / "coco", 1)

In [ ]:
import nncf
from typing import Dict


def transform_fn(data_item:Dict):
    """
    Quantization transform function. Extracts and preprocess input data from dataloader item for quantization.
    Parameters:
       data_item: Dict with data item produced by DataLoader during iteration
    Returns:
        input_tensor: Input data for quantization
    """
    input_tensor = det_validator.preprocess(data_item)['img'].numpy()
    return input_tensor


quantization_dataset = nncf.Dataset(det_data_loader, transform_fn)

For more accurate results, keep the operation in the postprocessing subgraph in floating point precision, using the ignored_scope parameter. You can visualize the model graph using https://netron.app for finding the names of nodes in in the postprocessing subgraph

In [ ]:
ignored_scope = nncf.IgnoredScope( # post-processing
    subgraphs=[
        nncf.Subgraph(inputs=['/model.22/Concat',
                              '/model.22/Concat_1',
                              '/model.22/Concat_2',
                              '/model.22/Concat_3',
                              '/model.22/Concat_4'],
                      outputs=['output0'])
    ]
)

# nncf.QuantizationPreset.MIXED symmetric quantization of weights and asymmetric quantization of activations.
quantized_det_model = nncf.quantize(
    ov_model,
    quantization_dataset,
    preset=nncf.QuantizationPreset.MIXED,
    ignored_scope=ignored_scope
)

In [ ]:
ov.save_model(quantized_det_model, "/content/models/yolov8n_openvino_int8_model/yolov8n_int8.xml",compress_to_fp16=False)